In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, matthews_corrcoef
from sklearn.preprocessing import LabelEncoder
from scipy import stats
import numpy as np
import joblib

from sklearn.preprocessing import LabelEncoder

In [2]:
# Load the CSV file
calibratedata = pd.read_csv('calibrateset_withVAEfeature.csv')
# Delete the "ID" and "Standard.SMILES" columns
calibratedata = calibratedata.drop(["ID", "SMILES"], axis=1)
X_calibratedata = calibratedata.drop("label", axis=1)
y_calibratedata = calibratedata["label"]
# Encode the labels
le = LabelEncoder()
y_calibratedata = le.fit_transform(y_calibratedata)

In [4]:
X_calib=X_calibratedata
y_calib=y_calibratedata

# Train a RandomForestClassifier (or any other classifier)
clf = joblib.load("XGboostbest_model.pkl")

# Apply the classifier to the calibration set and calculate p-values
y_pred_proba_calib = clf.predict_proba(X_calib)
#y_pred_proba_test = clf.predict_proba(X_test)

# Calculate the conformity scores
conformity_scores = y_pred_proba_calib[np.arange(len(y_calib)), y_calib]



In [8]:
def get_mcp_vector(confidence_level, testsettable):
    y_pred_proba_test = clf.predict_proba(testsettable)
    # Calculate the conformity scores
    conformity_scores = y_pred_proba_calib[np.arange(len(y_calib)), y_calib]

    # Calculate p-values for test set
    p_values = []
    for probas in y_pred_proba_test:
        instance_p_values = [np.mean(proba >= conformity_scores) for proba in probas]
        p_values.append(instance_p_values)
    p_values = np.array(p_values)
    alpha=confidence_level

    # Make predictions
    predictions = [(p > alpha).nonzero()[0] for p in p_values]
    filtered_indices = [i for i, prediction in enumerate(predictions) if len(prediction) == 1]
    # Create new vector based on filtered_indices
    new_vector = ["Reliable" if i in filtered_indices else "Unreliable" for i in range(len(y_pred_proba_test))]

    print(new_vector)
